## Descrição

**Definição**: Processo de identificação das alterações feitas nos dados na origem e
entrega dessas alterações ao destino

O Change Data Capture (CDC) é um conjunto de processos de software projetados para identificar, capturar e entregar alterações feitas em uma fonte de dados (geralmente bancos de dados transacionais como SQL Server) para sistemas de destino em tempo real. Em vez de realizar extrações completas e pesadas (full loads), o CDC monitora os logs de transação do banco para capturar apenas as inserções, atualizações e exclusões (INSERTs, UPDATEs e DELETEs). Isso permite manter um Lakehouse ou Data Warehouse sincronizado com latência mínima, reduzindo drasticamente a carga sobre os sistemas de origem e otimizando o tráfego de rede.

![](https://estuary.dev/static/93143bfefdc57eccf457e615cea3bfc3/a0efb/03_SQL_Change_Data_Capture_Trigger_Based_CDC_01b9232c38.png)


obs: Apesar da imagem estar em 2 tabelas separada (Source e Shadow Table), o processo pode ser feito na mesma tabela utilizando uma coluna de data e a outra com a operação efetuada(UPDATE, DELETE, INSERT).

## Exemplo de utilização

In [0]:
%sql
MERGE INTO target_talbe as t
USING source_table as s
ON t.key = s.key
WHEN MATCHED AND t.data < s.data
  THEN UPDATE SET *
WHEN MATCHED AND s.operacao = 'DELETE'
   THEN DELETE
WHEN NOT MATCHED
   THEN INSERT *

## Limitação

- O comando Merge não pode ser executado se múltiplas linhas de origem coincidirem e tentarem modificar a mesma linha de destino na tabela Delta.

- Um feed de CDC (Change Data Capture) com múltiplas atualizações para a mesma chave gerará uma exceção.

Para lidar com a limitação podemos utilizar a função rank e garantir que só será executada a ultima alteração. 

Como exemplo de código olhe os códigos da aplicação na pasta curso > 02.Processamento_de_dados > 2.1 - Change Data Capture